https://www.kaggle.com/code/pochenliu/linear-classifiers-xdawn-100-letter-accuracy

In [1]:
from tables import current_dir
!pip install mat4py
!pip install kagglehub

In [2]:
import kagglehub
import shutil
path = kagglehub.dataset_download("rramele/p300samplingdataset")
loc = "../p300samplingdataset/"
shutil.move(path, loc)
print("Path to dataset files: ", path)

100%|██████████| 308M/308M [00:19<00:00, 16.6MB/s] 

Extracting files...


Path to dataset files:  /home/dante/.cache/kagglehub/datasets/rramele/p300samplingdataset/versions/1


In [3]:
import sklearn as sk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
from mat4py import loadmat
import time
import glob
from sklearn.model_selection import train_test_split
from scipy.signal import welch, freqz, butter, filtfilt
from sklearn.model_selection import cross_val_predict, TimeSeriesSplit
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from mne.decoding import Vectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_val_score, StratifiedKFold, KFold
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.naive_bayes import  GaussianNB, MultinomialNB, BernoulliNB, CategoricalNB, ComplementNB

from mne import (io, compute_raw_covariance, read_events, pick_types, Epochs)
from mne.preprocessing import Xdawn
from mne.viz import plot_epochs_image
import mne
mne.set_log_level('WARNING')

from sklearn.pipeline import make_pipeline

import os

In [4]:
folder = '../p300samplingdataset/*.mat'
files = glob.glob(folder)
print(files)
print(len(files))

channels = ['Fz', 'Cz', 'P3', 'Pz', 'P4', 'P07', 'P08', 'Oz']

['../p300samplingdataset/P300S02.mat', '../p300samplingdataset/P300S04.mat', '../p300samplingdataset/P300S06.mat', '../p300samplingdataset/P300S08.mat', '../p300samplingdataset/P300S07.mat', '../p300samplingdataset/P300S01.mat', '../p300samplingdataset/P300S05.mat', '../p300samplingdataset/P300S03.mat']
8


In [5]:
def load_data(file):
    raw_data = loadmat(file)
    useful_data = raw_data['data'].copy()
    X = useful_data['X']
    Y = useful_data['y']
    T = useful_data['trial']
    F = useful_data['flash']
    
    return X, Y, T, F

In [6]:
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5): 
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y

In [7]:
from imblearn.under_sampling import RandomUnderSampler

# Cut the data into trials
def prepare_data(X, Y, flash, start, stop, filter_boo):
    X = np.array(X)

    if filter_boo:
        X = butter_bandpass_filter(X.T, low, high, fs).T

    start_samples = int(start*fs)
    stop_samples = int(stop*fs)

    X_samples = np.zeros((len(flash)-120, int(stop_samples-start_samples), 8))

    for i in range(len(flash)-120):
        event = flash[i][0]
        X_samples[i, :, :] = X[event+start_samples:event+stop_samples :]
    label     = [i[3] - 1 for i in flash]

    LIMIT = 4080 #the last trial is incomplete
    y = np.array(label[:LIMIT])
    X_samples = X_samples.reshape(X_samples.shape[0], X_samples.shape[1] * X_samples.shape[2])

    undersample = RandomUnderSampler(sampling_strategy='majority')
    X_under, y_under = undersample.fit_resample(X_samples, y)

    #     print(X_under.shape, y_under.shape)
    #     print(sum(y_under))

    return X_under, y_under


def prepare_data_nonUS(X, Y, flash, start, stop):
    X = np.array(X)

    start_samples = int(start*fs)
    stop_samples = int(stop*fs)

    X_samples = np.zeros((len(flash)-120, int(stop_samples-start_samples), 8))

    for i in range(len(flash)-120):
        event = flash[i][0] # should have 4080 flash events
        X_samples[i, :, :] = X[event+start_samples:event+stop_samples :] # should generate 4080 rows of 8 x 250 samples
    label     = [i[3] - 1 for i in flash] # hit = 1 nohit = 0, getting 4200 of them

    LIMIT = 4080 #the last trial is incomplete
    y = np.array(label[:LIMIT]) # taking the first 4080 out of these
    X_samples = X_samples.reshape(X_samples.shape[0], X_samples.shape[1] * X_samples.shape[2]) # reshape into 4080 rows of 8x250 = 2000 array

    # simply commented out
    #    undersample = RandomUnderSampler(sampling_strategy='majority')
    #    X_under, y_under = undersample.fit_resample(X_samples, y) # so retain in undersamples: equal numbers of hits & no hits

    #     print(X_under.shape, y_under.shape)
    #     print(sum(y_under))

    return X_samples, y

In [8]:
appX = []
appy = []

start = 0
stop = 1
fs = 250

for i in range(len(files)-7):
    file = files[i]
    X, Y, Trials, Flash = load_data(file)
    X_clean, y_clean = prepare_data(X, Y, Flash, start, stop, False)
    appX.append(X_clean)
    appy.append(np.array(y_clean))

In [ ]:
print(appX)